# eval on real questions

In [11]:
import pandas as pd
import os

folders = ['major','training_program', "uni"]
data = []
tmpdf = None
df = None
for folder in folders:
    path = f"../data/{folder}/test/"
    files = os.listdir(path=path)
    all_files = [os.path.join(path, f) for f in files]
    tmpdf = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
    print(tmpdf.shape)
    label = folder.replace("_program", "") if folder == "training_program" else folder
    tmpdf['label'] = label
    if df is None:
        df = tmpdf
    else:
        df = pd.concat([df, tmpdf], axis=0)
    print(len(files))

(115, 8)
39
(22, 8)
9
(20, 9)
11


In [2]:
df = df.sample(frac = 1)
df.shape

(157, 7)

In [3]:
import sys
sys.path.append("../")
from src.config import Configuration
from src.robot import RAGRobot
from src.service.provider import ProviderService

config = Configuration()

provider = ProviderService()
robot = RAGRobot(provider=provider)

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loaded index doc
loaded index doc
loaded index doc


In [4]:
df['prediction']= df['question'].apply(lambda x: robot.router.invoke({"question": x}).lower())

In [9]:
df.to_csv("../experiments/results/eval_routing.csv")

In [12]:
df = pd.read_csv("../experiments/results/eval_routing.csv")

In [13]:
labels = df['label'].to_list()
preds = df['prediction'].to_list()

from sklearn.metrics import classification_report

report = classification_report(y_true=labels, y_pred=preds)
print(report)

              precision    recall  f1-score   support

       major       0.97      0.74      0.84       115
       other       0.00      0.00      0.00         0
    training       0.40      0.64      0.49        22
         uni       0.79      0.75      0.77        20

    accuracy                           0.73       157
   macro avg       0.54      0.53      0.52       157
weighted avg       0.86      0.73      0.78       157



/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
with open("./intent_classification.txt", "w") as f:
    f.write(report)

# eval on random questions

In [1]:
import pandas as pd
import sys
sys.path.append("../")
from src.config import Configuration
from src.robot import RAGRobot
from src.service.provider import ProviderService


path = "../data/random_questions.csv"
df = pd.read_csv(path)

config = Configuration()

provider = ProviderService()
robot = RAGRobot(provider=provider)


/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loaded index doc
loaded index doc
loaded index doc


In [10]:
ai = provider.get_simple_gemini_pro()
t = ai.temperature
print(t)
t = ai.top_k
print(t)
t = ai.top_p
print(t)

0.0
None
None


In [4]:
df['prediction']= df['question'].apply(lambda x: robot.router.invoke({"question": x}).lower())


In [5]:
df.to_csv("../experiments/results/eval_routing_other.csv")


In [6]:
labels = df['label'].to_list()
preds = df['prediction'].to_list()

from sklearn.metrics import classification_report

report = classification_report(y_true=labels, y_pred=preds)
print(report)


              precision    recall  f1-score   support

       other       1.00      1.00      1.00        30

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

